In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from feat import Detector

### とりま

In [2]:
# from feat.plotting import imshow

# def plot_image_and_aus(image_path, aus):
#   """
# 画像と対応するアクションユニット（AU）の配列をプロットする関数。
# :param image_path: 画像のパス
# :param aus: アクションユニットの配列
# """
#   image = cv2.imread(image_path)
#   plt.figure(figsize=(10, 5))
#   plt.subplot(1, 2, 1)
#   plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#   plt.title("Image")
#   plt.axis("off")
#   plt.subplot(1, 2, 2)
#   plt.bar(range(len(aus)), list(aus.values()), tick_label=list(aus.keys()))
#   plt.title("Action Units")
#   plt.xlabel("AU")
#   plt.ylabel("Intensity")
#   plt.xticks(rotation=45)

#   plt.tight_layout()
#   plt.show()


# def detect_features_from_image(image_path):
#   #---

#   face_model = "retinaface"
#   landmark_model = "mobilefacenet" #"mobilefacenet"
#   #landmark_model = "pfld"
#   au_model = "svm"
#   au_model='xgb'
#   emotion_model = "resmasknet"
#   facepose_model="img2pose"
#   detector = Detector(
#     face_model=face_model,
#     landmark_model=landmark_model,
#     au_model=au_model,
#     emotion_model=emotion_model,
#     facepose_model=facepose_model
#   )

#   #detector

#   imshow(image_path)

#   #---
#   image = cv2.imread(image_path)
#   detected_faces = detector.detect_faces(image)
#   if not detected_faces:
#     return "検出された顔なし"

#   landmarks = detector.detect_landmarks(image, detected_faces)

#   aus = []
#   emotions = []
#   for i, face in enumerate(detected_faces):
#     au = detector.detect_aus(image, landmarks[i])
#     emotion = detector.detect_emotions(image, face, landmarks[i])
#     aus.append(au)
#     emotions.append(emotion)

#   return {
#       "face": detected_faces,
#       "landmarks": landmarks,
#       "aus": aus,
#       "emotions": emotions
#   }

# def get_aus_from_image(image_path):
#   detector = Detector(face_model="retinaface", landmark_model="mobilenet", au_model="svm")
#   image = cv2.imread(image_path)
#   try:
#     faces = detector.detect_faces(image)
#     if not faces:
#       return "No face detected"
#     landmarks = detector.detect_landmarks(image, faces)
#     aus = detector.detect_aus(image, landmarks)
#     if aus:
#       return aus[0]["aus"]
#     else:
#       return "No AU detected"
#   except Exception as e:
#     return str(e)

%%time

image_path = "./ForMachineLearning/20201127_1840_5_y/images/20201127_184657_040630.jpg"
print("-------------------")
features = detect_features_from_image(image_path)

aus = features["aus"]

print("-------------")
if aus and isinstance(aus[0], dict):
  plot_image_and_aus(image_path, aus[0])
else:
  print(aus)

### テスト

In [3]:
# %%time

# image_path = "./ForMachineLearning/20201127_1840_5_y/images/20201127_184657_040630.jpg"
# print("-------------------")
# #---

# face_model = "retinaface"
# landmark_model = "mobilefacenet" #"mobilefacenet"
# #landmark_model = "pfld"
# au_model = "svm"
# au_model='xgb'
# emotion_model = "resmasknet"
# facepose_model="img2pose"

# detector = Detector(
#   face_model=face_model,
#   landmark_model=landmark_model,
#   au_model=au_model,
#   emotion_model=emotion_model,
#   facepose_model=facepose_model
# )

# imshow(image_path)
# f = detector.detect_image(image_path)
# print(f.facebox)
# print(f.aus)
# print(f.emotions)
# print(f.facepose)

# figs = f.plot_detections(poses=True)


## AUなどの情報をJSONに書き出せるようにする

In [4]:
import os
import json
import numpy as np
from pathlib import Path


def calc_feat_in_images(images_directory):
  feats = []
  face_model = "retinaface"
  landmark_model = "mobilefacenet"
  au_model = "svm"
  au_model = 'xgb'
  emotion_model = "resmasknet"
  facepose_model = "img2pose"
  detector = Detector(
      face_model=face_model,
      landmark_model=landmark_model,
      au_model=au_model,
      emotion_model=emotion_model,
      facepose_model=facepose_model
  )
  for image_name in os.listdir(images_directory):
    if image_name.endswith('.jpg'):
      image_path = os.path.join(images_directory, image_name)
      try:
        result = detector.detect_image(image_path)
        feat = {
            "aus": result.aus.to_dict(),
            "emotions": result.emotions.to_dict(),
            "facepose": result.facepose.to_dict(),
            "facebox": result.facebox.to_dict(),
            "landmarks": result.landmarks.to_dict(),
        }
        feats.append([image_name, feat])
      except Exception as e:
        feats.append([image_name, 'Error', str(e)])
  return feats


def calc_feat_in_dir(for_ml_directory):
  results = {}
  for dir_name in os.listdir(for_ml_directory):
    images_directory = os.path.join(for_ml_directory, dir_name, 'images')
    if os.path.isdir(images_directory):
      feat_data = calc_feat_in_images(images_directory)
      results[dir_name] = feat_data
  return results

def save_feats_to_json(feats, filepath):
  with open(filepath, 'w') as fp:
    json.dump(feats, fp)

In [5]:
test_dir = 'ForMachineLearning/test_images_dir'

feat_data = calc_feat_in_images(test_dir)

save_feats_to_json(feat_data, 'feat_data_TEST.json')

100%|██████████| 1/1 [00:07<00:00,  7.01s/it]
/home/buntin/jupyter-env/lib64/python3.9/site-packages/feat/data.py:507: DeprecationWarning: Fex.facepose has now been renamed to Fex.poses
  warnings.warn(
/home/buntin/jupyter-env/lib64/python3.9/site-packages/feat/data.py:618: DeprecationWarning: Fex.facebox has been renamed to Fex.faceboxes
  warnings.warn(
100%|██████████| 1/1 [00:01<00:00,  1.20s/it]
/home/buntin/jupyter-env/lib64/python3.9/site-packages/feat/data.py:507: DeprecationWarning: Fex.facepose has now been renamed to Fex.poses
  warnings.warn(
/home/buntin/jupyter-env/lib64/python3.9/site-packages/feat/data.py:618: DeprecationWarning: Fex.facebox has been renamed to Fex.faceboxes
  warnings.warn(
100%|██████████| 1/1 [00:01<00:00,  1.22s/it]
/home/buntin/jupyter-env/lib64/python3.9/site-packages/feat/data.py:507: DeprecationWarning: Fex.facepose has now been renamed to Fex.poses
  warnings.warn(
/home/buntin/jupyter-env/lib64/python3.9/site-packages/feat/data.py:618: Deprec

In [6]:
# %%time
# image = cv2.imread(image_path)
# detected_faces = detector.detect_faces(image)
# if not detected_faces:
#   return "検出された顔なし"

# landmarks = detector.detect_landmarks(image, detected_faces)

# aus = []
# emotions = []
# for i, face in enumerate(detected_faces):
#   au = detector.detect_aus(image, landmarks[i])
#   emotion = detector.detect_emotions(image, face, landmarks[i])
#   aus.append(au)
#   emotions.append(emotion)
# print("-------------------")
# print(features)